In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import warnings
warnings.filterwarnings('ignore')
from time import time
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score

Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
attr = 'HS22'

In [10]:
df = pd.read_csv('Train2015-2018.csv', usecols=["TimeOfDay", attr], parse_dates=['TimeOfDay'])

In [11]:
df1 = pd.read_csv('Train2019JAN.csv', usecols=["TimeOfDay", attr], 
                    parse_dates=['TimeOfDay'])

In [12]:
df2 = pd.read_csv('Train2019FEB.csv', usecols=["TimeOfDay", attr], 
                    parse_dates=['TimeOfDay'])

In [13]:
df3 = pd.read_csv('Train2019MAR.csv', usecols=["TimeOfDay", attr], 
                    parse_dates=['TimeOfDay'])

In [14]:
df4 = pd.read_csv('Train2019APR.csv', usecols=["TimeOfDay", attr], 
                    parse_dates=['TimeOfDay'])

In [15]:
df = pd.concat([df,df1,df2,df3,df4])

In [16]:
df = df.set_index(['TimeOfDay'], drop=True)

In [17]:
df.dropna(subset=[attr], inplace=True)

In [18]:
split_date = pd.Timestamp('2019-01-01')

In [19]:
#df =  df[attr]
train_LSTM = df.loc[:split_date]
test_LSTM = df.loc[split_date:]

In [20]:
train = train_LSTM.values
test = test_LSTM.values
train = np.reshape(train, (-1, 1))
test = np.reshape(test, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
train_sc = scaler.fit_transform(train)
test_sc = scaler.transform(test)

In [ ]:
train_sc_df = pd.DataFrame(train_sc, columns=['HS22'], index=train_LSTM.index)
test_sc_df = pd.DataFrame(test_sc, columns=['HS22'], index=test_LSTM.index)

for s in range(1,2):
    train_sc_df['X_{}'.format(s)] = train_sc_df['HS22'].shift(s)
    test_sc_df['X_{}'.format(s)] = test_sc_df['HS22'].shift(s)

X_train = train_sc_df.dropna().drop('Y', axis=1)
y_train = train_sc_df.dropna().drop('X_1', axis=1)

X_test = test_sc_df.dropna().drop('Y', axis=1)
y_test = test_sc_df.dropna().drop('X_1', axis=1)

X_train = X_train.as_matrix()
y_train = y_train.as_matrix()

X_test = X_test.as_matrix()
y_test = y_test.as_matrix()

In [ ]:
X_train_lmse = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lmse = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print('Train shape: ', X_train_lmse.shape)
print('Test shape: ', X_test_lmse.shape)

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(7, input_shape=(1, X_train_lmse.shape[1]), activation='relu', kernel_initializer='lecun_uniform', return_sequences=False))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1)
history_lstm_model = lstm_model.fit(X_train_lmse, y_train, epochs=100, batch_size=1, verbose=1, shuffle=False, callbacks=[early_stop])

In [ ]:
y_pred_test_lstm = lstm_model.predict(X_test_lmse)
y_train_pred_lstm = lstm_model.predict(X_train_lmse)
print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(y_train, y_train_pred_lstm)))
print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(y_test, y_pred_test_lstm)))

In [ ]:
y_pred_test_lstm = scaler.inverse_transform(y_pred_test_lstm)

In [ ]:
y_test = scaler.inverse_transform(y_test)

In [ ]:
y_test_1 = np.concatenate(y_test)

In [ ]:
y_pred_test_lstm_1 = np.concatenate(y_pred_test_lstm)

In [ ]:
#data1 = test_LSTM.to_frame()

In [ ]:
test_LSTM.shape

In [ ]:
y_pred_test_lstm_1.shape

In [ ]:
y_pred_test_lstm_1 = list(y_pred_test_lstm_1)+[0]
test_LSTM['pred'] = y_pred_test_lstm_1

# data1.drop(data1.tail(1).index,inplace=True)

In [ ]:
test_LSTM.tail()

In [ ]:
test_LSTM[attr] = test_LSTM[attr].astype('float')

In [ ]:
test_LSTM['diff'] = np.absolute(test_LSTM[attr] - test_LSTM['pred'])

In [ ]:
test_LSTM.describe()